<a href="https://colab.research.google.com/github/chitransh0052/Neural-Network/blob/main/Hyper_parameter_tuning_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Hyper Parameter Tuning**

1. No. of Neurons in a layer.(Hidden layer)
2. No of hidden layers.
3. Activation functioni in hidden layer.
4. Dropout

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data=pd.read_csv('/content/diabetes.csv')

In [3]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [6]:
data.shape

(768, 9)

In [8]:
#seperate dependent and independent variable
X = data.drop('Outcome', axis=1)
y = data['Outcome']


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Data is ready to feed into th NN


In [14]:
!pip install keras-tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.8 MB/s eta 0:00:00


In [15]:
import kerastuner as kt

<ipython-input-15-5fd8096cdee5>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [23]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow import keras

In [36]:
def build_model(hp):
  model = Sequential()
  model.add(
            Dense(hp.Int("unit_0",min_value=8,max_value=128,step=8),
            activation=hp.Choice("activation_0",values=["sigmoid","relu","tanh"]),
            input_dim=8)
            )
  model.add(Dropout(hp.Choice("dropout_0",values=[0.10,0.20,0.30,0.40])))
  for i in range(1,hp.Int("num_layers",min_value=1,max_value=10)):
    model.add(
            Dense(hp.Int(f"unit_{i}",min_value=8,max_value=128,step=8),
            activation=hp.Choice(f"activation_{i}",values=["sigmoid","relu","tanh"]))
            )
    # Adding dropout layer within the loop, so 'i' is defined
    model.add(Dropout(hp.Choice(f"dropout_{i}",values=[0.10,0.20,0.30,0.40])))

  model.add(Dense(1,activation='sigmoid'))
  # Include 'accuracy' in the metrics to calculate and report it
  model.compile(loss='binary_crossentropy',optimizer=hp.Choice("optimizer", values=['adam','adagrad','rmsprop','adadelta']), metrics=['accuracy'])
  return model

In [37]:
tuner=kt.RandomSearch(build_model,
                objective="val_accuracy",
                max_trials=3,
                directory="chitransh",
                project_name="diabetes")

Reloading Tuner from chitransh/diabetes/tuner0.json


In [38]:
tuner.search(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 08s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 05m 08s


In [39]:
tuner.get_best_hyperparameters()[0].values

{'unit_0': 40,
 'activation_0': 'tanh',
 'dropout_0': 0.1,
 'num_layers': 6,
 'optimizer': 'adam',
 'unit_1': 32,
 'activation_1': 'sigmoid',
 'dropout_1': 0.3,
 'unit_2': 16,
 'activation_2': 'relu',
 'dropout_2': 0.4,
 'unit_3': 80,
 'activation_3': 'sigmoid',
 'dropout_3': 0.5,
 'unit_4': 16,
 'activation_4': 'tanh',
 'dropout_4': 0.4,
 'unit_5': 8,
 'activation_5': 'relu',
 'dropout_5': 0.1,
 'unit_6': 72,
 'activation_6': 'sigmoid',
 'dropout_6': 0.1,
 'unit_7': 24,
 'activation_7': 'sigmoid',
 'dropout_7': 0.4}

In [40]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [41]:
model.fit(X_train,y_train,initial_epoch=11,epochs=100,validation_data=(X_test,y_test))

Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.6169 - loss: 0.6643 - val_accuracy: 0.6429 - val_loss: 0.6513
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6331 - loss: 0.6685 - val_accuracy: 0.6429 - val_loss: 0.6480
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6289 - loss: 0.6641 - val_accuracy: 0.6429 - val_loss: 0.6452
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6349 - loss: 0.6388 - val_accuracy: 0.6429 - val_loss: 0.6431
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6571 - loss: 0.6480 - val_accuracy: 0.6429 - val_loss: 0.6362
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6727 - loss: 0.6449 - val_accuracy: 0.6429 - val_loss: 0.6261
Epoch 18/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6442 - loss: 0.6374 - val_accuracy: 0.6429 - val_loss: 0.6199
Epoch 19/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6539 - loss: 0.6269 - val_accuracy: 0